In [2]:
%env THEANO_FLAGS=device=cuda0,floatX=float32

from layers import *
from lasagne.layers import *
import lasagne

from lasagne.nonlinearities import LeakyRectify

from collections import OrderedDict

import theano
import theano.tensor as T

from nets import ResGenerator, Discriminator

from theano.tensor.shared_randomstreams import RandomStreams
srng = RandomStreams(seed=234)

env: THEANO_FLAGS=device=cuda0,floatX=float32


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
/tmp/try_flags_b4btBv.c:4:19: fatal error: cudnn.h: Нет такого файла или каталога
compilation terminated.

Mapped name None to device cuda0: GeForce GTX 1060 6GB (0000:01:00.0)


## Define da Nets!

In [3]:
# a & b
realA = T.tensor4("Real A input")
realB = T.tensor4("Real B input")

# G(a) -> b_hat & F(b) -> a_hat
genA2B = ResGenerator(filter_size=(128, 128), input_var=realA)
genB2A = ResGenerator(filter_size=(128, 128), input_var=realB)

# D_a(F(b)) & D_b(G(a))
discA = Discriminator(genB2A, wasserstein=True, real_inp_var=realA)
discB = Discriminator(genA2B, wasserstein=True, real_inp_var=realB)

# F(G(a)) -> a_cyc & G(F(b)) -> b_cyc
cycleA = genB2A.get_output(genA2B.output_var)
cycleB = genA2B.get_output(genB2A.output_var)

## Losses

In [4]:
# TODO: clean this mess up 

class WGANLosses:
    def __init__(self, gen, disc, lambd=10):
        self.epsilon = srng.uniform([1]) # low=0, high=1
        
        x_hat = self.epsilon * disc.real_inp_var + (1 - self.epsilon) * gen.output_var
        
        x_hat_score = get_output(gen.layers["out"], x_hat).mean()
        x_hat_grad = T.grad(x_hat_score, x_hat)[0]
        
        wgan_loss = (disc.real_out.mean() - disc.fake_out.mean())
        gradient_penalty = ((T.sqrt(T.sum(x_hat_grad**2)) - 0.01)**2).mean()

        self.disc_loss = -wgan_loss + lambd * gradient_penalty
        self.gen_loss = -disc.fake_out.mean()

In [17]:
# Hyperparams
ADAM_PARAMS = {"learning_rate" : 1e-4, "beta1" : 0.5, "beta2" : 0.9}
LAMBD = 100

In [18]:
# |a - F(G(a))| + |b - G(F(b))|
cycle_loss = (T.abs_(realA - cycleA) + T.abs_(realB - cycleB)).mean()

# mess
b_loss = WGANLosses(genA2B, discB)
a_loss = WGANLosses(genB2A, discA)

# wasserstein-gp losses + cycle loss
gen_comb_loss = a_loss.gen_loss + b_loss.gen_loss + cycle_loss * LAMBD
disc_comb_loss = a_loss.disc_loss + b_loss.disc_loss

# updates with combined weights 
gen_updates = lasagne.updates.adam(gen_comb_loss, genA2B.params + genB2A.params, **ADAM_PARAMS)
disc_updates = lasagne.updates.adam(disc_comb_loss, discA.params + discB.params, **ADAM_PARAMS)

trainGen = theano.function([realA, realB], gen_comb_loss, updates=gen_updates)
trainDisc = theano.function([realA, realB], disc_comb_loss, updates=disc_updates)

In [ ]:
pics2array()